# Sistema de Recomendación: Vectorización de Usuarios
Personalización del vector académico de cada estudiante según sus asignaturas relevantes.

In [1]:
import pandas as pd
import numpy as np
import pickle
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Cargar datos de la encuesta
df_usuario = (
    pd.read_excel('usuarios/Encuesta recien graduados - pregrado.xlsx')
    .loc[:, [
        'carrera',
        '6.1 ¿Qué asignaturas te resultaron más relevantes durante la carrera? Para completar esta pregunta realizarlo separando por comas: Ejemplo: Estática, Mecánica de materiales, Dinámica'
    ]]
    .rename(columns={
        'carrera': 'Carrera',
        '6.1 ¿Qué asignaturas te resultaron más relevantes durante la carrera? Para completar esta pregunta realizarlo separando por comas: Ejemplo: Estática, Mecánica de materiales, Dinámica': 'Asignaturas Relevantes'
    })
)

# Cargar vectores académicos procesados
with open('datos_procesados.pkl', 'rb') as f:
    datos = pickle.load(f)
tfidf_epn_69d = datos['tfidf_epn_69d']
grupos_bge_ngram = datos['grupos_bge_ngram']

# Diccionario de mapeo carreras
mapeo_carreras = {
    '(RRA20) COMPUTACIÓN': 'Ingenieria En Ciencias De La Computacion',
    '(RRA20) AGROINDUSTRIA': 'Ingenieria Agroindustria',
    '(RRA20) ADMINISTRACIÓN DE EMPRESAS': 'Licenciatura Administracion De Empresas',
    '(RRA20) INGENIERÍA AMBIENTAL': 'Ingenieria Ambiental',
    '(RRA20) ECONOMÍA': 'Economia',
    'INGENIERIA EN CIENCIAS ECONOMICAS Y FINANCIERAS': 'Economia',
    '(RRA20) ELECTRICIDAD': 'Ingenieria En Electricidad',
    '(RRA20) ELECTRÓNICA Y AUTOMATIZACIÓN': 'Ingenieria En Electronica Y Automatizacion',
    '(RRA20) FÍSICA': 'Fisica',
    'FISICA': 'Fisica',
    '(RRA20) GEOLOGÍA': 'Ingenieria En Geologia',
    'INGENIERIA GEOLOGICA': 'Ingenieria En Geologia',
    '(RRA20) INGENIERÍA DE LA PRODUCCIÓN': 'Ingenieria De La Produccion',
    '(RRA20) MATEMÁTICA': 'Matematica',
    '(RRA20) MECÁNICA': 'Ingenieria En Mecanica',
    'INGENIERIA MECANICA': 'Ingenieria En Mecanica',
    '(RRA20) PETRÓLEOS': 'Ingenieria En Petroleos',
    '(RRA20) INGENIERÍA QUÍMICA': 'Ingenieria Quimica',
    '(RRA20) DESARROLLO DE SOFTWARE': 'Ingenieria En Software',
    '(RRA20) SOFTWARE': 'Ingenieria En Software',
    '(RRA20) TELECOMUNICACIONES': 'Ingenieria En Telecomunicaciones',
    '(RRA20) INGENIERÍA CIVIL': 'Ingenieria Civil'
}

print(f"Usuarios cargados: {len(df_usuario)}")
print("Dimensiones de habilidades (filas):", tfidf_epn_69d.shape[0])
print("Carreras (columnas):", tfidf_epn_69d.shape[1])
print("Nombres de dimensiones:", list(tfidf_epn_69d.index)[:5], "...")

Usuarios cargados: 380
Dimensiones de habilidades (filas): 69
Carreras (columnas): 24
Nombres de dimensiones: ['administración de empresas, gestión de calidad, gestión de datos ...', 'agroindustria, agronomía', 'análisis de datos, análisis de materiales, análisis de sistemas ...', 'aprendizaje no supervisado, aprendizaje supervisado', 'arcgis, qgis'] ...


## Asignación de vectores académicos
Se asigna a cada estudiante su vector académico estándar y se personaliza según sus asignaturas relevantes.

In [2]:
# Asignar carrera vectorizada a cada usuario
df_usuario['Carrera Asignada'] = df_usuario['Carrera'].map(mapeo_carreras)
df_usuario['Vector Academico'] = df_usuario['Carrera Asignada'].apply(
    lambda x: tfidf_epn_69d.T.loc[x].values if pd.notnull(x) else None
)

# Funciones auxiliares
habilidades = list(tfidf_epn_69d.T.columns)
vectorizer = TfidfVectorizer().fit(habilidades)

def normalizar_texto(texto):
    return [a.strip().lower() for a in re.split(r',|;|/|\n', str(texto)) if a.strip()] if pd.notnull(texto) else []

def encontrar_habilidades_similares(asignatura, threshold=0.5):
    vect_asig = vectorizer.transform([asignatura])
    vect_habs = vectorizer.transform(habilidades)
    sims = cosine_similarity(vect_asig, vect_habs).flatten()
    return [habilidades[i] for i in np.where(sims >= threshold)[0]]

def asignar_ponderacion(vector_base, asignaturas_relevantes):
    if vector_base is None or not isinstance(vector_base, np.ndarray):
        return None
    vector = vector_base.copy()
    for asignatura in normalizar_texto(asignaturas_relevantes):
        for hab in encontrar_habilidades_similares(asignatura):
            vector[habilidades.index(hab)] = 0.99
    return vector

# Crear vector personalizado
df_usuario['Vector Estudiante Profesional'] = df_usuario.apply(
    lambda row: asignar_ponderacion(row['Vector Academico'], row['Asignaturas Relevantes']),
    axis=1
)

print(f"Vectores personalizados creados: {df_usuario['Vector Estudiante Profesional'].notna().sum()}")

Vectores personalizados creados: 335


El **335** se refiere a los estudiantes que tienen un vector personalizado distinto al vector académico base porque sí seleccionaron asignaturas relevantes y estas fueron mapeadas a habilidades técnicas. Los 45 estudiantes restantes (380 - 335) no seleccionaron asignaturas relevantes o sus *asignaturas no coincidieron con ninguna habilidad técnica*, por lo que conservan solo el vector académico base de su carrera.

## Análisis de cambios vectoriales
Identificación de estudiantes cuyo vector fue modificado y en qué dimensiones.

In [3]:
# Comparar vectores y registrar cambios
df_usuario['Dimensiones Modificadas'] = df_usuario.apply(
    lambda row: np.where(np.abs(row['Vector Academico'] - row['Vector Estudiante Profesional']) > 1e-6)[0]
    if isinstance(row['Vector Academico'], np.ndarray) and isinstance(row['Vector Estudiante Profesional'], np.ndarray) else np.array([]),
    axis=1
)

df_modificados = df_usuario[df_usuario['Dimensiones Modificadas'].apply(len) > 0]

print(f"Estudiantes con vector modificado: {len(df_modificados)} / {len(df_usuario)}")
print(f"Porcentaje: {len(df_modificados) / len(df_usuario) * 100:.1f}%")

df_usuario[['Carrera', 'Asignaturas Relevantes', 'Dimensiones Modificadas']].head(10)

Estudiantes con vector modificado: 264 / 380
Porcentaje: 69.5%


,Carrera,Asignaturas Relevantes,Dimensiones Modificadas
0,(RRA20) INGENIERÍA CIVIL,Hidráulica,[]
1,(RRA20) ELECTRÓNICA Y AUTOMATIZACIÓN,"Control industrial, PLCs, instalaciones eléctr...","[14, 15, 36]"
2,(RRA20) SOFTWARE,"Programación, aplicaciones móviles, estructura...","[19, 28]"
3,(RRA20) ELECTRÓNICA Y AUTOMATIZACIÓN,Robótica y todo lo relacionado a PLCs,[]
4,(RRA20) DESARROLLO DE SOFTWARE,"Bases de datos, Desarrollo de aplicaciones web...","[8, 19, 28, 62]"
5,(RRA20) ELECTRÓNICA Y AUTOMATIZACIÓN,Electrónica,[26]
6,(RRA20) DESARROLLO DE SOFTWARE,"programación,bases de datos, gestión de proyectos","[0, 8, 28, 62]"
7,(RRA20) INGENIERÍA CIVIL,Diseño de Proyectos Estructurales,[23]
8,(RRA20) INGENIERÍA DE LA PRODUCCIÓN,Optimización de procesos,[51]
9,(RRA20) INGENIERÍA DE LA PRODUCCIÓN,"Gestión de procesos, calidad, logística y dist...",[0]


## Incorporación de habilidades blandas
Se agregan 7 dimensiones de habilidades blandas (evaluadas 1-5) a los vectores existentes de 69 dimensiones, para un total de 76 dimensiones.
- Se mapea las 7 preguntas del Excel con sus columnas exactas
- Extrae los valores de escala 1-5 y los normaliza a rango [0, 1]
- **Las 7 dimensiones:** Gestión, Comunicación efectiva, Liderazgo, Trabajo en equipo, Ética profesional, Responsabilidad social, Aprendizaje autónomo

In [4]:
# Mapeo de preguntas de habilidades blandas en el archivo Excel
habilidades_blandas_config = {
    'Gestión': 'A continuación, por favor evalúa qué tan bien consideras que desarrollaste las siguientes habilidades blandas durante tu formación en la EPN. Usa una escala de 1 a 5, donde 1 = Nada desarrollado y 5 = Totalmente desarrollado. [Gestión: Gestionar actividades complejas, como proyectos de ingeniería, en el campo de estudio.]',
    'Comunicación efectiva': 'A continuación, por favor evalúa qué tan bien consideras que desarrollaste las siguientes habilidades blandas durante tu formación en la EPN. Usa una escala de 1 a 5, donde 1 = Nada desarrollado y 5 = Totalmente desarrollado. [Comunicación efectiva: Presentar y defender proyectos técnicos en español e inglés en entornos profesionales.]',
    'Liderazgo': 'A continuación, por favor evalúa qué tan bien consideras que desarrollaste las siguientes habilidades blandas durante tu formación en la EPN. Usa una escala de 1 a 5, donde 1 = Nada desarrollado y 5 = Totalmente desarrollado. [Liderazgo: Dirigir equipos, organizando tareas y tomando decisiones clave.]',
    'Trabajo en equipo': 'A continuación, por favor evalúa qué tan bien consideras que desarrollaste las siguientes habilidades blandas durante tu formación en la EPN. Usa una escala de 1 a 5, donde 1 = Nada desarrollado y 5 = Totalmente desarrollado. [Trabajo en equipo: Participar activamente en equipos para resolver problemas de ingeniería.]',
    'Ética profesional': 'A continuación, por favor evalúa qué tan bien consideras que desarrollaste las siguientes habilidades blandas durante tu formación en la EPN. Usa una escala de 1 a 5, donde 1 = Nada desarrollado y 5 = Totalmente desarrollado. [Ética profesional: Actuar con responsabilidad ética en la toma de decisiones técnicas, respetando normas y compromisos legales.]',
    'Responsabilidad social': 'A continuación, por favor evalúa qué tan bien consideras que desarrollaste las siguientes habilidades blandas durante tu formación en la EPN. Usa una escala de 1 a 5, donde 1 = Nada desarrollado y 5 = Totalmente desarrollado. [Responsabilidad social: Considerar aspectos jurídicos, culturales y de sostenibilidad al desarrollar soluciones de ingeniería.]',
    'Aprendizaje autónomo': 'A continuación, por favor evalúa qué tan bien consideras que desarrollaste las siguientes habilidades blandas durante tu formación en la EPN. Usa una escala de 1 a 5, donde 1 = Nada desarrollado y 5 = Totalmente desarrollado. [Aprendizaje autónomo: Buscar activamente aprendizaje continuo y desarrollar habilidades profesionales de manera autónoma.]'
}

df_encuesta_completa = pd.read_excel('usuarios/Encuesta recien graduados - pregrado.xlsx')

def extraer_calificacion(valor):
    if pd.isnull(valor):
        return None
    valor_str = str(valor).strip()
    primer_digito = re.search(r'[1-5]', valor_str)
    if primer_digito:
        return int(primer_digito.group())
    return None

# Solo extraer y normalizar, pero NO dejar las columnas blandas en df_usuario
skills_blandas_temp = {}
for skill, col_name in habilidades_blandas_config.items():
    if col_name in df_encuesta_completa.columns:
        calificaciones = df_encuesta_completa[col_name].apply(extraer_calificacion)
        skills_blandas_temp[skill] = (calificaciones - 1) / 4
    else:
        print(f"Advertencia: Columna '{col_name}' no encontrada en Excel")
        skills_blandas_temp[skill] = 0.0

# Almacenar las 7 dimensiones blandas como arrays para cada usuario
skills_blandas_matrix = np.vstack([skills_blandas_temp[skill] for skill in habilidades_blandas_config.keys()]).T

df_usuario['skills_blandas_array'] = list(skills_blandas_matrix)

In [5]:
# Eliminar columnas de habilidades blandas individuales si existen
df_usuario = df_usuario.drop(columns=[c for c in habilidades_blandas_config.keys() if c in df_usuario.columns], errors='ignore')

## Ampliación de vectores a 76 dimensiones
Se concatenan los vectores de 69 dimensiones técnicas con las 7 dimensiones de habilidades blandas.

In [6]:
# Crear vectores expandidos con 76 dimensiones (69 técnicas + 7 blandas)
def expandir_vector_76d(vector_69d, skills_blandas_array):
    if vector_69d is None or skills_blandas_array is None:
        return None
    return np.concatenate([vector_69d, skills_blandas_array])

# Aplicar expansión al vector académico
df_usuario['Vector Academico 76d'] = df_usuario.apply(
    lambda row: expandir_vector_76d(row['Vector Academico'], row['skills_blandas_array']),
    axis=1
)

def asignar_ponderacion_76d(vector_base, asignaturas_relevantes):
    if vector_base is None or vector_base.shape[0] != 76:
        return None
    vector = vector_base.copy()
    for asignatura in normalizar_texto(asignaturas_relevantes):
        for hab in encontrar_habilidades_similares(asignatura):
            idx = habilidades.index(hab)
            vector[idx] = 0.99
    return vector

df_usuario['Vector Estudiante 76d'] = df_usuario.apply(
    lambda row: asignar_ponderacion_76d(row['Vector Academico 76d'], row['Asignaturas Relevantes']),
    axis=1
)

# Eliminar las columnas auxiliares después de crear los vectores expandidos
df_usuario = df_usuario.drop(columns=['skills_blandas_array', 'Vector Academico 76d'], errors='ignore')

print(f"Vectores expandidos a 76 dimensiones:")
print(f"  - 69 dimensiones técnicas")
print(f"  - 7 dimensiones blandas: {list(habilidades_blandas_config.keys())}")
print(f"Vectores creados: {df_usuario['Vector Estudiante 76d'].notna().sum()}")

Vectores expandidos a 76 dimensiones:
  - 69 dimensiones técnicas
  - 7 dimensiones blandas: ['Gestión', 'Comunicación efectiva', 'Liderazgo', 'Trabajo en equipo', 'Ética profesional', 'Responsabilidad social', 'Aprendizaje autónomo']
Vectores creados: 335


In [7]:
# Visualización de ejemplos de vectores completos (76 dimensiones)
num_ejemplos = 2
vectores_validos = df_usuario['Vector Estudiante 76d'].apply(lambda v: v is not None and v.shape[0] == 76)
ejemplos = df_usuario[vectores_validos].head(num_ejemplos)

print(f"\nEjemplos de vectores de estudiantes (76 dimensiones):\n")
for idx, row in ejemplos.iterrows():
    v = row['Vector Estudiante 76d']
    print(f"Estudiante {idx} | Carrera: {row['Carrera']}")
    print(f"  Técnicas (0-68): {np.round(v[:69], 3)}")
    print(f"  Blandas  (69-75): {np.round(v[69:], 3)}")
    print(f"  Etiquetas blandas: {list(habilidades_blandas_config.keys())}")
    print('-'*80)

print("\nLas primeras 69 dimensiones corresponden a habilidades técnicas, las últimas 7 a habilidades blandas en este orden:")
for i, skill in enumerate(habilidades_blandas_config.keys()):
    print(f"  {69+i}: {skill}")


Ejemplos de vectores de estudiantes (76 dimensiones):

Estudiante 0 | Carrera: (RRA20) INGENIERÍA CIVIL
  Técnicas (0-68): [0.276 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.    0.977 0.    0.    0.    1.    0.    0.    0.
 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.    0.131 0.069 0.    0.    0.    0.    0.    0.    0.
 0.    1.    0.    0.    0.    0.    0.    0.    0.   ]
  Blandas  (69-75): [1. 1. 1. 1. 1. 1. 1.]
  Etiquetas blandas: ['Gestión', 'Comunicación efectiva', 'Liderazgo', 'Trabajo en equipo', 'Ética profesional', 'Responsabilidad social', 'Aprendizaje autónomo']
--------------------------------------------------------------------------------
Estudiante 1 | Carrera: (RRA20) ELECTRÓNICA Y AUTOMATIZACIÓN
  Técnicas (0-68): [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
 0.    0.    0.99  0.99  0.    0.

In [8]:
df_usuario.head()

,Carrera,Asignaturas Relevantes,Carrera Asignada,Vector Academico,Vector Estudiante Profesional,Dimensiones Modificadas,Vector Estudiante 76d
0,(RRA20) INGENIERÍA CIVIL,Hidráulica,Ingenieria Civil,"[0.27593165009980686, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.27593165009980686, 0.0, 0.0, 0.0, 0.0, 0.0,...",[],"[0.27593165009980686, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,(RRA20) ELECTRÓNICA Y AUTOMATIZACIÓN,"Control industrial, PLCs, instalaciones eléctr...",Ingenieria En Electronica Y Automatizacion,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[14, 15, 36]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,(RRA20) SOFTWARE,"Programación, aplicaciones móviles, estructura...",Ingenieria En Software,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[19, 28]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,(RRA20) ELECTRÓNICA Y AUTOMATIZACIÓN,Robótica y todo lo relacionado a PLCs,Ingenieria En Electronica Y Automatizacion,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,(RRA20) DESARROLLO DE SOFTWARE,"Bases de datos, Desarrollo de aplicaciones web...",Ingenieria En Software,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.99,...","[8, 19, 28, 62]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.99,..."


## Sistema de Recomendación: Similitud de Coseno por Carrera
Cálculo de similitud entre vectores de estudiantes (76d) y ofertas laborales (76d) **de la misma carrera**.
Solo recomienda ofertas laborales relevantes para la carrera del estudiante.
### Mapeo de carreras a archivos CSV
Carga ofertas laborales específicas por carrera del estudiante:

In [9]:
# Mapeo de carreras EPN a carpetas y archivos CSV
carrera_to_csv = {
    'Ingenieria En Ciencias De La Computacion': 'todas_las_plataformas/Computación/Computación_Merged.csv',
    'Ingenieria Agroindustria': 'todas_las_plataformas/Agroindustria/Agroindustria_Merged.csv',
    'Licenciatura Administracion De Empresas': 'todas_las_plataformas/Administración_de_Empresas/Administración_de_Empresas_Merged.csv',
    'Ingenieria Ambiental': 'todas_las_plataformas/Ingeniería_Ambiental/Ingeniería_Ambiental_Merged.csv',
    'Economia': 'todas_las_plataformas/Economía/Economía_Merged.csv',
    'Ingenieria En Electricidad': 'todas_las_plataformas/Electricidad/Electricidad_Merged.csv',
    'Ingenieria En Electronica Y Automatizacion': 'todas_las_plataformas/Electrónica_y_Automatización/Electrónica_y_Automatización_Merged.csv',
    'Fisica': 'todas_las_plataformas/Física/Física_Merged.csv',
    'Ingenieria En Geologia': 'todas_las_plataformas/Geología/Geología_Merged.csv',
    'Ingenieria De La Produccion': 'todas_las_plataformas/Ingeniería_de_la_Producción/Ingeniería_de_la_Producción_Merged.csv',
    'Ingenieria En Materiales': 'todas_las_plataformas/Materiales/Materiales_Merged.csv',
    'Ingenieria En Mecanica': 'todas_las_plataformas/Mecánica/Mecánica_Merged.csv',
    'Ingenieria En Mecatronica': 'todas_las_plataformas/Mecatrónica/Mecatrónica_Merged.csv',
    'Ingenieria En Petroleos': 'todas_las_plataformas/Petróleos/Petróleos_Merged.csv',
    'Ingenieria Quimica': 'todas_las_plataformas/Ingeniería_Química/Ingeniería_Química_Merged.csv',
    'Ingenieria En Telecomunicaciones': 'todas_las_plataformas/Telecomunicaciones/Telecomunicaciones_Merged.csv',
    'Ingenieria Civil': 'todas_las_plataformas/Ingeniería_Civil/Ingeniería_Civil_Merged.csv',
    'Matematica': 'todas_las_plataformas/Matemática/Matemática_Merged.csv',
    'Matematica Aplicada': 'todas_las_plataformas/Matemática_Aplicada/Matemática_Aplicada_Merged.csv',
    'Ingenieria En Software': 'todas_las_plataformas/Software/Software_Merged.csv',
    'Ingenieria En Ciencias De Datos': 'todas_las_plataformas/Ciencia_de_Datos/Ciencia_de_Datos_Merged.csv',
    'Ingenieria En Sistemas De Informacion': 'todas_las_plataformas/Sistemas_de_Información/Sistemas_de_Información_Merged.csv',
}

print(f'Mapeo de carreras a archivos CSV cargado ({len(carrera_to_csv)} carreras disponibles)')

Mapeo de carreras a archivos CSV cargado (22 carreras disponibles)


### Función para cargar ofertas de una carrera específica
Carga y vectoriza las ofertas laborales de una carrera particular usando sus vectores 76d (69 técnicas + 7 blandas).

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import os

def cargar_y_vectorizar_ofertas_carrera(ruta_csv, habilidades_whitelist, grupos_bge_ngram):
    """Carga ofertas de un CSV y las vectoriza a 76 dimensiones"""
    if not os.path.exists(ruta_csv):
        return None, None
    
    df_ofertas = pd.read_csv(ruta_csv, dtype=str)
    if not {'skills', 'description', 'EURACE_skills'}.issubset(df_ofertas.columns):
        return None, None
    
    # Vectorizar habilidades técnicas (69d)
    textos = df_ofertas[['skills', 'description']].fillna('').agg(' '.join, axis=1).str.lower().tolist()
    vectorizer = CountVectorizer(vocabulary=habilidades_whitelist, analyzer='word', ngram_range=(1, 5), lowercase=True)
    X = vectorizer.fit_transform(textos)
    matriz_td = pd.DataFrame(X.T.toarray(), index=vectorizer.get_feature_names_out())
    
    # Agrupar por dimensiones
    matriz_69d = pd.DataFrame(0, index=grupos_bge_ngram.keys(), columns=range(len(textos)))
    for label, terms in grupos_bge_ngram.items():
        terms_validos = [t for t in terms if t in matriz_td.index]
        if terms_validos:
            matriz_69d.loc[label] = matriz_td.loc[terms_validos].sum(axis=0)
    
    # Aplicar TF-IDF
    tfidf = TfidfTransformer(norm='l2')
    tfidf_69d = tfidf.fit_transform(matriz_69d.values)
    tfidf_69d_df = pd.DataFrame(tfidf_69d.toarray(), index=matriz_69d.index, columns=matriz_69d.columns).T
    
    return tfidf_69d_df, df_ofertas

# Cargar habilidades whitelist
with open('datos_procesados.pkl', 'rb') as f:
    datos = pickle.load(f)
habilidades_whitelist = datos['habilidades']

print('Función de vectorización de ofertas por carrera cargada')

Función de vectorización de ofertas por carrera cargada


### Generar recomendaciones por carrera del estudiante
Para cada estudiante, carga las ofertas de su carrera específica y calcula similitud de coseno.
Muestra ranking con: Cargo, Similitud, Descripción y Habilidades Requeridas.

In [11]:
# Sistema de recomendación por carrera
def obtener_recomendaciones_carrera(idx_estudiante, top_n=5):
    """Obtiene ofertas recomendadas para un estudiante de su misma carrera"""
    estudiante = df_usuario.iloc[idx_estudiante]
    carrera_asignada = estudiante['Carrera Asignada']
    vector_estudiante_76d = estudiante['Vector Estudiante 76d']
    
    # Validaciones
    if not carrera_asignada or carrera_asignada not in carrera_to_csv:
        return None, None
    if vector_estudiante_76d is None or len(vector_estudiante_76d) != 76:
        return None, None
    
    # Cargar ofertas de la carrera
    ruta_csv = carrera_to_csv[carrera_asignada]
    tfidf_ofertas_69d, df_ofertas = cargar_y_vectorizar_ofertas_carrera(ruta_csv, habilidades_whitelist, grupos_bge_ngram)
    
    if tfidf_ofertas_69d is None or tfidf_ofertas_69d.empty:
        return None, None
    
    # Concatenar habilidades blandas de ofertas con sus vectores técnicos
    vectores_ofertas_76d = []
    for idx in range(len(df_ofertas)):
        vector_tecnico = tfidf_ofertas_69d.iloc[idx].values
        vector_blandas = np.array([
            1 if 'gestion' in str(df_ofertas.iloc[idx]['EURACE_skills']).lower() else 0,
            1 if 'comunicacion' in str(df_ofertas.iloc[idx]['EURACE_skills']).lower() else 0,
            1 if 'liderazgo' in str(df_ofertas.iloc[idx]['EURACE_skills']).lower() else 0,
            1 if 'equipo' in str(df_ofertas.iloc[idx]['EURACE_skills']).lower() else 0,
            1 if 'etica' in str(df_ofertas.iloc[idx]['EURACE_skills']).lower() else 0,
            1 if 'responsabilidad' in str(df_ofertas.iloc[idx]['EURACE_skills']).lower() else 0,
            1 if 'aprendizaje' in str(df_ofertas.iloc[idx]['EURACE_skills']).lower() else 0,
        ])
        vector_76d = np.concatenate([vector_tecnico, vector_blandas])
        vectores_ofertas_76d.append(vector_76d)
    
    vectores_ofertas_76d = np.array(vectores_ofertas_76d)
    
    # Calcular similitud de coseno
    similitudes = cosine_similarity([vector_estudiante_76d], vectores_ofertas_76d)[0]
    
    # Ordenar por similitud (mayor a menor)
    indices_ordenados = np.argsort(similitudes)[::-1]
    
    # Seleccionar top N ofertas ÚNICAS (sin duplicados por título de cargo)
    resultado = []
    cargos_vistas = set()  # Rastrear títulos de cargo ya incluidos
    
    for idx_oferta in indices_ordenados:
        if len(resultado) >= top_n:
            break
        
        # Evitar duplicados por título del cargo
        cargo_titulo = df_ofertas.iloc[idx_oferta]['job_title']
        if cargo_titulo not in cargos_vistas:
            cargos_vistas.add(cargo_titulo)
            resultado.append({
                'Rank': len(resultado) + 1,
                'Similitud': similitudes[idx_oferta],
                'Cargo': cargo_titulo,
                'Descripción': str(df_ofertas.iloc[idx_oferta]['description'])[:60] + '...',
                'EURACE Skills': df_ofertas.iloc[idx_oferta]['EURACE_skills']
            })
    
    return pd.DataFrame(resultado), carrera_asignada

# Mostrar recomendaciones para los primeros 3 estudiantes
print('\n' + '='*120)
print('SISTEMA DE RECOMENDACIÓN: RANKING DE OFERTAS POR CARRERA DEL ESTUDIANTE')
print('='*120 + '\n')

habilidades_blandas_etiquetas = ['Gestión', 'Comunicación efectiva', 'Liderazgo', 'Trabajo en equipo', 
                                   'Ética profesional', 'Responsabilidad social', 'Aprendizaje autónomo']

for idx_est in range(min(3, len(df_usuario))):
    estudiante = df_usuario.iloc[idx_est]
    vector_76d = estudiante['Vector Estudiante 76d']
    
    if vector_76d is None or len(vector_76d) != 76:
        continue
    
    print(f"\nEstudiante {idx_est} | Carrera: {estudiante['Carrera']}")
    print(f"Asignaturas Relevantes: {estudiante['Asignaturas Relevantes']}")
    print(f"Habilidades Blandas:")
    for i, skill in enumerate(habilidades_blandas_etiquetas):
        ponderacion = vector_76d[69 + i]
        print(f"  - {skill}: {ponderacion:.3f}")
    
    recomendaciones_df, carrera = obtener_recomendaciones_carrera(idx_est, top_n=5)
    
    if recomendaciones_df is None:
        print(f"No hay ofertas disponibles para esta carrera")
    else:
        print(f"\nTop 5 Ofertas Recomendadas ({carrera}):")
        for idx, row in recomendaciones_df.iterrows():
            print(f"\n  Rank {row['Rank']}: Similitud {row['Similitud']:.4f}")
            print(f"    Cargo a Desempeñar: {row['Cargo']}")
            #print(f"    Descripción: {row['Descripción']}")
            print(f"    EURACE Skills: {row['EURACE Skills']}")
    
    print('\n' + '-'*120)


SISTEMA DE RECOMENDACIÓN: RANKING DE OFERTAS POR CARRERA DEL ESTUDIANTE


Estudiante 0 | Carrera: (RRA20) INGENIERÍA CIVIL
Asignaturas Relevantes: Hidráulica
Habilidades Blandas:
  - Gestión: 1.000
  - Comunicación efectiva: 1.000
  - Liderazgo: 1.000
  - Trabajo en equipo: 1.000
  - Ética profesional: 1.000
  - Responsabilidad social: 1.000
  - Aprendizaje autónomo: 1.000

Top 5 Ofertas Recomendadas (Ingenieria Civil):

  Rank 1: Similitud 0.6751
    Cargo a Desempeñar: Civil Engineer
    EURACE Skills: Gestión, Liderazgo, Trabajo en equipo, Ética profesional, Responsabilidad social, Aprendizaje autónomo

  Rank 2: Similitud 0.6366
    Cargo a Desempeñar: Sr. Project Manager, Capital Construction
    EURACE Skills: Gestión, Comunicación efectiva, Liderazgo, Trabajo en equipo, Ética profesional, Responsabilidad social, Aprendizaje autónomo

  Rank 3: Similitud 0.6294
    Cargo a Desempeñar: Construction Superintendent
    EURACE Skills: Gestión, Liderazgo, Trabajo en equipo, Responsab